## PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook
for the following questions.


In [15]:
# Establish connection with SQLite Database
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///data/country_club_db.sqlite')


### QUESTIONS:
**Q10**: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!
